In [20]:
# TO DO:
# call from csv for specific pokemon + stats
# call from csv for specific moves
# Physical Special split
# Typing for Type modifier
# debug damage calculation!!!
# add 0 EVs
# Ensure total EVs don't exceed 508
# IV modifier??
# level modifier?

from itertools import product
import math
from Typing import typing

# def_pokemon_type = "normal"
# atk_type = "normal"

atk_base = (100, 165, 150, 85, 130, 50) #Calyrex Ice Rider
def_base = (114, 85, 70, 85, 80, 30) #Amoongus

# Define nature effects
natures = {
    'Adamant': (1.0, 1.1, 1.0, 0.9, 1.0, 1.0),
    'Bashful': (1.0, 1.0, 1.0, 1.0, 1.0, 1.0), #Neutral
    'Bold': (1.0, 0.9, 1.1, 1.0, 1.0, 1.0),
    'Brave': (1.0, 1.1, 1.0, 1.0, 1.0, 0.9),
    'Calm': (1.0, 0.9, 1.0, 1.0, 1.1, 1.0),
    'Careful': (1.0, 1.0, 1.0, 0.9, 1.1, 1.0),
    'Docile': (1.0, 1.0, 1.0, 1.0, 1.0, 1.0),  # Neutral
    'Gentle': (1.0, 1.0, 0.9, 1.0, 1.1, 1.0),
    'Hardy': (1.0, 1.0, 1.0, 1.0, 1.0, 1.0),  # Neutral
    'Hasty': (1.0, 1.0, 0.9, 1.0, 1.0, 1.1),
    'Impish': (1.0, 1.0, 1.1, 0.9, 1.0, 1.0),
    'Jolly': (1.0, 1.0, 1.0, 0.9, 1.0, 1.1),
    'Lax': (1.0, 1.0, 1.1, 1.0, 0.9, 1.0),
    'Lonely': (1.0, 1.1, 0.9, 1.0, 1.0, 1.0),
    'Mild': (1.0, 1.0, 0.9, 1.1, 1.0, 1.0),
    'Modest': (1.0, 0.9, 1.0, 1.1, 1.0, 1.0),
    'Naive': (1.0, 1.0, 1.0, 1.0, 0.9, 1.1),
    'Naughty': (1.0, 1.1, 1.0, 1.0, 0.9, 1.0),
    'Quiet': (1.0, 1.0, 1.0, 1.1, 1.0, 0.9),
    'Quirky': (1.0, 1.0, 1.0, 1.0, 1.0, 1.0),  # Neutral
    'Rash': (1.0, 1.0, 1.0, 1.1, 0.9, 1.0),
    'Relaxed': (1.0, 1.0, 1.1, 1.0, 1.0, 0.9),
    'Sassy': (1.0, 1.0, 1.0, 1.0, 1.1, 0.9),
    'Serious': (1.0, 1.0, 1.0, 1.0, 1.0, 1.0),  # Neutral
    'Timid': (1.0, 0.9, 1.0, 1.0, 1.0, 1.1)
}

def apply_nature(stats, nature):
    nature_modifiers = natures[nature]
    return tuple(math.floor(stat * nature_mod) for stat, nature_mod in zip(stats, nature_modifiers))

IV = 31
level = 50
atk_nature = 'Brave'
def_nature = 'Bold'
base_power = 120
Spread = 'True'
Crit = 'False'
STAB = "True"
Type = 2.0

# Calculate stats
def calculate_stats(base, EV, nature):
    stats = []
    for i in range(6):
        if i == 0:  # HP
            stat = math.floor(((2 * base[i] + IV + math.floor(EV[i] / 4)) * level) / 100) + level + 10
        else:
            stat = math.floor(((2 * base[i] + IV + math.floor(EV[i] / 4)) * level) / 100) + 5
        stats.append(stat)
    return apply_nature(stats, nature)

# atk_pokemon_stats = calculate_stats(atk_base, atk_EV, atk_nature)
# def_pokemon_stats = calculate_stats(def_base, def_EV, def_nature)

# print("Attacker Stats:", atk_pokemon_stats)
# print("Defender Stats:", def_pokemon_stats)

# Custom rounding method (round down at 0.5)
def custom_round(x):
    """Custom rounding method to round down at 0.5"""
    return int(x + 0.5) if x - int(x) >= 0.5 else int(x)

# Calculate damage
def damage_calc(level, base_power, atk_pokemon_stats, def_pokemon_stats):
    # Calculate the intermediate values, rounding each multiplication to the nearest integer
    damage = (((2 * level )/ 5 + 2) * base_power * (calculate_stats(atk_base, atk_EV, atk_nature)[1] / calculate_stats(def_base, def_EV, def_nature)[2]) / 50 + 2)
    return damage

# Calculate spread modifier
def spread_modifer(level, base_power, atk_pokemon_stats, def_pokemon_stats):
    damage = math.floor(damage_calc(level, base_power, atk_pokemon_stats, def_pokemon_stats))
    if Spread == 'True':
        spread_modifer = 0.75
    else:
        spread_modifer = 1
    return damage * spread_modifer

# Calculate crit modifier
def crit_modifer(level, base_power, atk_pokemon_stats, def_pokemon_stats):
    damage = math.floor(spread_modifer(level, base_power, atk_pokemon_stats, def_pokemon_stats))
    if Crit == 'True':
        crit_modifer = 1.5
    else:
        crit_modifer = 1
    return damage * crit_modifer

# Iterate through all possible EV combinations for attacker

results = []
step = 8  # Change this to the step value you desire (e.g., 4, 8, 12, etc.)

# Iterate through all possible EV combinations for attacker
for hp_evs in range(4, 256, step):
    for def_evs in range(4, 256, step):
        atk_EV = (4, 252, 0, 0, 0, 252)
        def_EV = (hp_evs, 0, def_evs, 0, 0, 0)

        # Calculate stats for attacking Pokémon
        atk_pokemon_stats = calculate_stats(atk_base, atk_EV, atk_nature)

        # Calculate stats for defending Pokémon
        def_pokemon_stats = calculate_stats(def_base, def_EV, def_nature)

        damage_rolls = []
        for i in range(85, 101):  # Multiply by 0.85 to 1, inclusive
            multiplier = i / 100  # Convert the range to a float multiplier
            damage = math.floor(crit_modifer(level, base_power, atk_pokemon_stats[1], def_pokemon_stats[2]) * multiplier)
            damage_rolls.append(damage)

        # Apply STAB modifier rounding
        if STAB == 'True':
            STAB_modifier = 1.5
        else:
            STAB_modifier = 1
        damage_rolls = [math.floor(damage * STAB_modifier) for damage in damage_rolls]

        # Apply Type modifier rounding
        damage_rolls = [math.floor(damage * Type) for damage in damage_rolls]

        #Find min and max damage rolls
        Min_roll = min(damage_rolls)
        Max_roll = max(damage_rolls)

        # print(atk_EV[1], str(atk_nature), "Urshifu (Single Strike) Wicked Blow vs.",def_EV[0],"HP /",def_EV[2], "Def", str(def_nature), "Flutter Mane on a critical hit:",damage_rolls[0],"-",damage_rolls[-1], "(",round(Min_roll/calculate_stats(def_base, def_EV, def_nature)[0] * 100, 1),"% -",round(Max_roll/calculate_stats(def_base, def_EV, def_nature)[0] * 100, 1),"%)")
        # print(damage_rolls)
        # print("Total EVs:", (hp_evs + def_evs))
        
        result = {"HP EVs": hp_evs, "Def EVs": def_evs,"Total_EVs": (hp_evs + def_evs), "Damage Rolls": damage_rolls, "Max Roll": round(Max_roll/calculate_stats(def_base, def_EV, def_nature)[0] * 100, 1)}
        results.append(result)
        # print(results)

def threshold(result):
    if result["Max Roll"] < 100:
        return result
    else:
        return None

filtered_results = list(filter(threshold, results))

# Find the result with the lowest "Total_EVs"
lowest_total_evs_result = min(filtered_results, key=lambda x: x["Total_EVs"])

# Print the result with the lowest "Total_EVs"
print("Lowest Total EVs:", lowest_total_evs_result)

        # print(f"{hp_evs}, {def_evs}")
    # print(f"EVs: {def_EV}, Stats: {atk_pokemon_stats}, Damage Rolls: {damage_rolls}")


# Calculate damage rolls
# damage_rolls = []
# for i in range(85, 101):  # Multiply by 0.85 to 1, inclusive
#     multiplier = i / 100  # Convert the range to a float multiplier
#     damage = math.floor(damage_calc(level, base_power, atk_pokemon_stats[1], def_pokemon_stats[2]) * multiplier)
#     damage_rolls.append(damage)

# # Print min and max damage rolls
# Min_roll = min(damage_rolls)
# Max_roll = max(damage_rolls)

# print(atk_EV[1], str(atk_nature), "Urshifu (Single Strike) Wicked Blow vs.",def_EV[0],"HP /",def_EV[2], "Def", str(def_nature), "Flutter Mane on a critical hit:",damage_rolls[0],"-",damage_rolls[-1], "(",round(Min_roll/calculate_stats(def_base, def_EV, def_nature)[0] * 100, 1),"% -",round(Max_roll/calculate_stats(def_base, def_EV, def_nature)[0] * 100, 1),"%)")
# print(damage_rolls)


Lowest Total EVs: {'HP EVs': 188, 'Def EVs': 252, 'Total_EVs': 440, 'Damage Rolls': [180, 182, 182, 186, 188, 188, 192, 194, 198, 198, 200, 204, 204, 206, 210, 212], 'Max Roll': 99.5}
